In [ ]:
# Library
import numpy as np
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import DictionaryLearning, sparse_encode
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

np.set_printoptions(threshold=np.inf)
np.set_printoptions(suppress=True)

# Load data
measurement = np.load('../../dataset/meas_symm_1.npz', allow_pickle=False)
header, data = measurement['header'], measurement['data']
data_cir = data['cirs'][:1000]  # Using 1000 samples for simplicity

# Split data
trainCIR, testCIR = train_test_split(data_cir, test_size=0.2, random_state=42)

# Define channels
alice_channel = 3  # Channel 3 is ALICE (legitimate)
eve_channel = 6    # Channel 6 is EVE (illegitimate)

# Set the number of dictionary atoms (components) and sparsity level
# N_components = 3  # Number of Sparse Components
n_components = 5 # Adjust this number based on your data
n_nonzero_coefs = 3  # Number of non-zero coefficients in sparse coding

In [ ]:
train_alice_cirs = trainCIR[:, alice_channel, :, :]  # Alice's CIRs
train_eve_cirs = trainCIR[:, eve_channel, :, :]      # Eve's CIRs
train_alice_magnitude = np.abs(train_alice_cirs[..., 0] + 1j * train_alice_cirs[..., 1])
train_eve_magnitude = np.abs(train_eve_cirs[..., 0] + 1j * train_eve_cirs[..., 1])
train_data_combined  = np.vstack((train_alice_magnitude, train_eve_magnitude))

# Learn the dictionary
dict_learner = DictionaryLearning(n_components=n_components, transform_algorithm='lasso_lars', transform_n_nonzero_coefs=n_nonzero_coefs)
# dict_learner.fit(train_data_combined)
# D = dict_learner.components_  # Shape: (n_components, feature_dim)


train_alice_sparse_codes = dict_learner.fit_transform(train_alice_magnitude)
train_eve_sparse_codes = dict_learner.fit_transform(train_eve_magnitude)
train_sparse_codes = np.vstack((train_alice_sparse_codes, train_eve_sparse_codes))

In [ ]:
def create_sparse_representation(data, n_components=5, n_nonzero_coefs=2):
    """
    Create a sparse representation of CIR data
    
    Parameters:
    - data: Input data with shape (n_samples, n_data_points, n_features)
    - n_components: Number of sparse components to use
    - n_nonzero_coefs: Number of non-zero coefficients to keep
    
    Returns:
    - Sparse representation of the data
    """
    # Original data shape: (100, 251, 2)
    n_samples, n_data_points, n_features = data.shape

    # Initialize sparse representation array
    sparse_representation = np.zeros((n_samples, n_components, n_features))

    # Process each feature separately
    for feature in range(n_features):
        # Extract data for current feature
        feature_data = data[:, :, feature]  # Shape: (n_samples, n_data_points)
        
        # Create a random dictionary (you might want to learn this from data)
        dictionary = np.random.randn(n_components, n_data_points)  # Shape: (n_components, n_data_points)
        
        # Initialize Sparse Coder
        sparse_coder = SparseCoder(
            dictionary=dictionary, 
            transform_algorithm='lasso_lars',  # LASSO regression for sparsity
            transform_n_nonzero_coefs=n_nonzero_coefs
        )
        
        # Fit and transform the data to get sparse representation
        sparse_codes = sparse_coder.transform(feature_data)  # Shape: (n_samples, n_components)
        
        # Store the sparse codes for this feature
        sparse_representation[:, :, feature] = sparse_codes  # Shape: (n_samples, n_components)

    return sparse_representation


# ----------------------------------------------------- Preprocessing -----------------------------------------------------
# ----------------- Training data -----------------
train_alice_cirs = trainCIR[:, alice_channel, :, :]  # Alice's CIRs
train_eve_cirs = trainCIR[:, eve_channel, :, :]      # Eve's CIRs
train_data_combined  = np.vstack((train_alice_cirs, train_eve_cirs))
print('train_data_combined.shape:', train_data_combined.shape)

# Create sparse representation
train_sparse_cirs = create_sparse_representation(train_data_combined)
print('train_sparse_cirs.shape:', train_sparse_cirs.shape)

In [ ]:
import numpy as np

# Assume your dataset has shape (100, 251, 2)
data = np.random.randn(100, 251, 2)  # Replace this with your actual CIR dataset

# Extract the real and imaginary parts
real_part = data[..., 0]  # Shape: (100, 251)
imaginary_part = data[..., 1]  # Shape: (100, 251)

# Calculate the magnitude
magnitude = np.sqrt(real_part**2 + imaginary_part**2)  # Shape: (100, 251)

# Add an additional dimension to make it (100, 251, 1)
magnitude = magnitude[..., np.newaxis]  # Shape: (100, 251, 1)

# Print the result shape to verify
print("Magnitude shape:", magnitude[0])


In [ ]:
import numpy as np
from sklearn.decomposition import SparseCoder

def create_sparse_representation(data, n_components=5, n_nonzero_coefs=2):
    """
    Create a sparse representation of CIR data
    
    Parameters:
    - data: Input data with shape (n_samples, n_data_points, n_features)
    - n_components: Number of sparse components to use
    - n_nonzero_coefs: Number of non-zero coefficients to keep
    
    Returns:
    - Sparse representation of the data
    """
    # Original data shape: (100, 251, 2)
    n_samples, n_data_points, n_features = data.shape

    # Initialize sparse representation array
    sparse_representation = np.zeros((n_samples, n_components, n_features))

    # Process each feature separately
    for feature in range(n_features):
        # Extract data for current feature
        feature_data = data[:, :, feature]  # Shape: (n_samples, n_data_points)
        
        # Create a random dictionary (you might want to learn this from data)
        dictionary = np.random.randn(n_components, n_data_points)  # Shape: (n_components, n_data_points)
        
        # Initialize Sparse Coder
        sparse_coder = SparseCoder(
            dictionary=dictionary, 
            transform_algorithm='lasso_lars',  # LASSO regression for sparsity
            transform_n_nonzero_coefs=n_nonzero_coefs
        )
        
        # Fit and transform the data to get sparse representation
        sparse_codes = sparse_coder.transform(feature_data)  # Shape: (n_samples, n_components)
        
        # Store the sparse codes for this feature
        sparse_representation[:, :, feature] = sparse_codes  # Shape: (n_samples, n_components)

    return sparse_representation

# Example usage
# Generate random data representing CIR signals
data = np.random.randn(100, 251, 2)  # 100 CIR samples, 251 data points per CIR, 2 features (real and imaginary)

# Create sparse representation
sparse_data = create_sparse_representation(data)

# Print shapes to verify
print("Original data shape:", data.shape)
print("Sparse representation shape:", sparse_data.shape)


In [ ]:
import numpy as np
from sklearn.decomposition import SparseCoder

def create_sparse_representation(data, n_components=5, n_nonzero_coefs=2):
    """
    Create a sparse representation of multidimensional data
    
    Parameters:
    - data: Input data with shape (n_samples, n_features, n_channels)
    - n_components: Number of sparse components to use
    - n_nonzero_coefs: Number of non-zero coefficients to keep
    
    Returns:
    - Sparse representation of the data
    """
    # Original data shape: (100, 251, 2)
    n_samples, n_features, n_channels = data.shape
    
    # Initialize sparse representation array
    sparse_representation = np.zeros((n_samples, n_components, n_channels))
    
    # Process each channel separately
    for channel in range(n_channels):
        # Extract data for current channel
        channel_data = data[:, :, channel]
        
        # Reshape data to 2D for sparse coding
        # Shape changes from (100, 251) to (100, 251)
        X = channel_data.reshape(-1, n_features)
        
        # Create a random dictionary (you might want to learn this from data)
        dictionary = np.random.randn(n_components, n_features)
        
        # Initialize Sparse Coder
        sparse_coder = SparseCoder(
            dictionary=dictionary, 
            transform_algorithm='lasso_lars',  # LASSO regression for sparsity
            transform_n_nonzero_coefs=n_nonzero_coefs
        )
        
        # Fit and transform the data to get sparse representation
        sparse_codes = sparse_coder.transform(X)
        
        # Store the sparse codes for this channel
        sparse_representation[:, :, channel] = sparse_codes.reshape(n_samples, n_components)
    
    return sparse_representation

# Example usage
# Generate random data
data = np.random.randn(100, 251, 2)

# Create sparse representation
sparse_data = create_sparse_representation(data)

# Print shapes to verify
print("Original data shape:", data.shape)
print("Sparse representation shape:", sparse_data.shape)

In [ ]:
# ----------------------------------------------------- Preprocessing -----------------------------------------------------
# ----------------- Training data -----------------
train_alice_cirs = trainCIR[:, alice_channel, :, :]  # Alice's CIRs
train_alice_real = train_alice_cirs[..., 0]  # Shape: (100, 251)
train_alice_imaginary = train_alice_cirs[..., 1]  # Shape: (100, 251)
train_alice_magnitude = np.sqrt(np.maximum(train_alice_real**2 + train_alice_imaginary**2, 0))
train_alice_magnitude = train_alice_magnitude[..., np.newaxis]
# train_alice_magnitude = np.abs(train_alice_cirs[..., 0] + 1j * train_alice_cirs[..., 1])

train_eve_cirs = trainCIR[:, eve_channel, :, :]      # Eve's CIRs
train_eve_real = train_alice_cirs[..., 0]  # Shape: (100, 251)
train_eve_imaginary = train_alice_cirs[..., 1]  # Shape: (100, 251)
train_eve_magnitude = np.sqrt(np.maximum(train_eve_real**2 + train_eve_imaginary**2, 0))
train_eve_magnitude = train_eve_magnitude[..., np.newaxis]

train_data_combined  = np.vstack((train_alice_magnitude, train_eve_magnitude))
train_sparse_cir = create_sparse_representation(train_data_combined, n_components, n_nonzero_coefs)
print(train_sparse_cir.shape)
# # Learn the dictionary
# dict_learner = DictionaryLearning(n_components=n_components, transform_algorithm='lasso_lars', transform_n_nonzero_coefs=n_nonzero_coefs)
# dict_learner.fit(train_data_combined)
# # D = dict_learner.components_  # Shape: (n_components, feature_dim)


# train_alice_sparse_codes = dict_learner.transform(train_alice_magnitude)
# train_eve_sparse_codes = dict_learner.transform(train_eve_magnitude)
# train_sparse_codes = np.vstack((train_alice_sparse_codes, train_eve_sparse_codes))
# print(train_alice_sparse_codes.shape)

In [ ]:
# ----------------- Test data -----------------
test_alice_cirs = testCIR[:, alice_channel, :, :]
test_alice_real = test_alice_cirs[..., 0]  # Shape: (100, 251)
test_alice_imaginary = test_alice_cirs[..., 1]  # Shape: (100, 251)
test_alice_magnitude = np.sqrt(np.maximum(test_alice_real**2 + test_alice_imaginary**2, 0))
test_alice_magnitude = test_alice_magnitude[..., np.newaxis]

test_eve_cirs = testCIR[:, eve_channel, :, :]
test_eve_real = test_eve_cirs[..., 0]  # Shape: (100, 251)
test_eve_imaginary = test_eve_cirs[..., 1]  # Shape: (100, 251)
test_eve_magnitude = np.sqrt(np.maximum(test_eve_real**2 + test_eve_imaginary**2, 0))
test_eve_magnitude = test_eve_magnitude[..., np.newaxis]

# test amplitude
# test_alice_magnitude = np.abs(test_alice_CIRs[..., 0] + 1j * test_alice_CIRs[..., 1])
# test_eve_magnitude = np.abs(test_eve_CIRs[..., 0] + 1j * test_eve_CIRs[..., 1]) 
test_cirs_combined = np.vstack((test_alice_magnitude, test_eve_magnitude))
test_sparse_cirs = create_sparse_representation(test_cirs_combined, n_components, n_nonzero_coefs)
print(test_sparse_cirs.shape)
# Transform test data into sparse codes
# test_alice_sparse_codes = dict_learner.transform(test_alice_magnitude)
# test_eve_sparse_codes = dict_learner.transform(test_eve_magnitude)

# # Combine the sparse codes
# test_sparse_codes = np.vstack((test_alice_sparse_codes, test_eve_sparse_codes))
# print('Test data shape:', test_sparse_codes.shape)

In [ ]:
# Labels for training data
train_alice_labels = np.zeros(test_alice_cirs.shape[0])  # Label '0' for Alice.
train_eve_labels = np.ones(test_eve_cirs.shape[0])       # Label '1' for Eve.
train_labels = np.hstack((train_alice_labels, train_eve_labels))
print(train_labels.shape)

# Labels for test data
test_alice_labels = np.zeros(test_alice_cirs.shape[0])  # Label '0' for Alice.
test_eve_labels = np.ones(test_eve_cirs.shape[0])       # Label '1' for Eve.
test_labels = np.hstack((test_alice_labels, test_eve_labels))
print(test_labels.shape)


In [ ]:
# Dictionary
atoms = train_sparse_cir.reshape(train_sparse_cir.shape[0], -1)
D = atoms.T
print('Dictionary shape:', D.shape)

In [ ]:
import numpy as np
from sklearn.decomposition import SparseCoder

def create_sparse_representation(data, n_components=5, n_nonzero_coefs=2):
    """
    Create a sparse representation of multidimensional data
    
    Parameters:
    - data: Input data with shape (n_samples, n_features, n_channels)
    - n_components: Number of sparse components to use
    - n_nonzero_coefs: Number of non-zero coefficients to keep
    
    Returns:
    - Sparse representation of the data
    """
    # Original data shape: (100, 251, 2)
    n_samples, n_features, n_channels = data.shape
    
    # Initialize sparse representation array
    sparse_representation = np.zeros((n_samples, n_components, n_channels))
    
    # Process each channel separately
    for channel in range(n_channels):
        # Extract data for current channel
        channel_data = data[:, :, channel]
        
        # Reshape data to 2D for sparse coding
        # Shape changes from (100, 251) to (100, 251)
        X = channel_data.reshape(-1, n_features)
        
        # Create a random dictionary (you might want to learn this from data)
        dictionary = np.random.randn(n_components, n_features)
        
        # Initialize Sparse Coder
        sparse_coder = SparseCoder(
            dictionary=dictionary, 
            transform_algorithm='lasso_lars',  # LASSO regression for sparsity
            transform_n_nonzero_coefs=n_nonzero_coefs
        )
        
        # Fit and transform the data to get sparse representation
        sparse_codes = sparse_coder.transform(X)
        
        # Store the sparse codes for this channel
        sparse_representation[:, :, channel] = sparse_codes.reshape(n_samples, n_components)
    
    return sparse_representation

# Example usage
# Generate random data
data = np.random.randn(100, 251, 2)

# Create sparse representation
sparse_data = create_sparse_representation(data)

# Print shapes to verify
print("Original data shape:", data.shape)
print("Sparse representation shape:", sparse_data.shape)

In [ ]:
import numpy as np
from sklearn.decomposition import SparseCoder

def create_sparse_representation(data, n_components=5, n_nonzero_coefs=2):
    """
    Create a sparse representation of CIR data
    
    Parameters:
    - data: Input data with shape (n_samples, n_data_points, n_features)
    - n_components: Number of sparse components to use
    - n_nonzero_coefs: Number of non-zero coefficients to keep
    
    Returns:
    - Sparse representation of the data
    """
    # Original data shape: (100, 251, 2)
    n_samples, n_data_points, n_features = data.shape

    # Initialize sparse representation array
    sparse_representation = np.zeros((n_samples, n_components, n_features))

    # Process each feature separately
    for feature in range(n_features):
        # Extract data for current feature
        feature_data = data[:, :, feature]  # Shape: (n_samples, n_data_points)
        
        # Create a random dictionary (you might want to learn this from data)
        dictionary = np.random.randn(n_components, n_data_points)  # Shape: (n_components, n_data_points)
        
        # Initialize Sparse Coder
        sparse_coder = SparseCoder(
            dictionary=dictionary, 
            transform_algorithm='lasso_lars',  # LASSO regression for sparsity
            transform_n_nonzero_coefs=n_nonzero_coefs
        )
        
        # Fit and transform the data to get sparse representation
        sparse_codes = sparse_coder.transform(feature_data)  # Shape: (n_samples, n_components)
        
        # Store the sparse codes for this feature
        sparse_representation[:, :, feature] = sparse_codes  # Shape: (n_samples, n_components)

    return sparse_representation

# Example usage
# Generate random data representing CIR signals
data = np.random.randn(100, 251, 2)  # 100 CIR samples, 251 data points per CIR, 2 features (real and imaginary)

# Create sparse representation
sparse_data = create_sparse_representation(data)

# Print shapes to verify
print("Original data shape:", data.shape)
print("Sparse representation shape:", sparse_data.shape)
